In [2]:
!pip install -q kagglehub
import kagglehub

# Download latest version
path = kagglehub.dataset_download("shubhammaindola/harry-potter-books")

print("Path to dataset files:", path)

/Users/jigyanshupati/nanogpt/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 2.28M/2.28M [00:01<00:00, 1.29MB/s]

Extracting files...
Path to dataset files: /Users/jigyanshupati/.cache/kagglehub/datasets/shubhammaindola/harry-potter-books/versions/1


In [8]:
with open("/Users/jigyanshupati/nanogpt/01 Harry Potter and the Sorcerers Stone.txt") as file:
    books = file.read()
len(books)

439478

In [12]:
characters = sorted(list(set(books)))
vocab_size = len(characters)
print("Number of unique characters:", vocab_size)

Number of unique characters: 82


In [171]:
#create mapping from int to char and char to int
int_to_char = {i: c for i, c in enumerate(characters)}
char_to_int = {c: i for i, c in enumerate(characters)}
import tiktoken
enc = tiktoken.get_encoding("o200k_base")
assert enc.decode(enc.encode("hello world")) == "hello world"

# To get the tokeniser corresponding to a specific model in the OpenAI API:
enc = tiktoken.encoding_for_model("gpt-4o")
#function to convert text to integers
def encode(text):
    return enc.encode(text)
#function to convert integers to text
def decode(ints):
    return enc.decode(ints)

decode(encode("hello world"))

'hello world'

In [172]:
!pip install torch torchvision torchaudio
import torch
data= torch.tensor(encode(books), dtype=torch.long)
data.shape

torch.Size([104922])

In [173]:
train_size = int(0.9 * len(data))
train_data = data[:train_size]
val_data = data[train_size:]

In [174]:
block_size = 8
train_data[:block_size+1]


tensor([   44,   428,    13,   326, 31127,    13,   415,  1628,  4356])

In [175]:
#batch size
batch_size = 4
block_size = 8
torch.manual_seed(42)
def get_batch(split):
    #generate random batch of data
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y
xb, yb = get_batch('train')


    

In [176]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(42)
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size, block_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        self.position_embedding_table = nn.Embedding(block_size, vocab_size)
        self.lm_head = nn.Linear(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=idx.device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        logits = self.lm_head(x) # (B,T,C)
        if targets is not None:
            B, T, C = logits.shape
            logits = logits.view(B*T, C) # (B*T,C)
            targets = targets.view(B*T) # (B*T,)
            loss = F.cross_entropy(logits, targets) # (B*T,)
            return logits, loss
        else:
            return logits
        
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]  # (B, T)
            logits = self(idx_cond)           # get only logits here
            logits = logits[:, -1, :]         # focus on last time step
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx
m=BigramLanguageModel(vocab_size,block_size)
# logits,loss=m(xb, yb)  # -2.1813e+00
decode(m.generate(torch.zeros((1, 1), dtype=torch.long), 1000)[0].tolist())


'!WQ0[9K>)M;TL)MBa*9#f0f$iaoP;ff!F@5@!dWS<@P5NEXP^b9^/NF@eo=Aq$,B-Qmc-f%1dEof;,1b;"#@<GKm3+TJn:R5:6nLJ#r#gM]#l]MoC@SI=q$*qfoj=6f*])1_e@<E;5!JPI,iRL@Q<]oY@(*#>Hf@hfGfo9g>QJ1KBR]J=EQ#@8eWbq1i7]eD3D<]mZQ#=DB3P]]3@[+P-<RQ\\EV"i#?qgBi:;Agcf58!`!B!?QX;<@?"?BTl9_-BY-@D\\f0_G.I]qP%-6<,4T!=qrMB++o?_gq&X]!347<eDh<-/TEq$;ZfoVE0Rbn<1];OB;.]Q!Q%JE.mqAf=q**#]#fH;$$D\\]a5Q8TmZKTk](QGP"9MBo%]ZXE$NKfSJWA3*-mY>5@<C$D#"&/5%(\\"3#D3pm)]]3G81,ff+(6!a("#]<Ba\'B3ao@;`d%P`=L@\'D%,qMGKce]n*;/K5%0Q9m:MQ3GB0q1Gq.V5X8a.Q03jG>4fBSBQeW<@(#ef_,2To-iN?Q!>,!!Ki6.l]<I\\,8<&g5.@r.G*?9.b&i#lpQqG-EPQ]_/Oq!"?]K>YrOQNgZ<,B\\Zq-h\\@h"PBA7A"Q"f9CWea-//rr513/)fSc6F5]3o%pq3_A[1DK+p>d%Ba8qrA5i5"=DT*$j!S>@N<q@<p@c,!!?_8)qD61@N3C^>3F<Neb.:"Q#!7oQ3n1O+`MP2\\]P<1H1ZX](qI68@<@<1]]1@p(0V<@<9K>SBMR?od3#fOGMBf%JpLTL(I$#@BF-q$nr?QZ1:451a*?%]Ma;fm!#W:5eS*i29n*oT%Te^ED!!abOo\\P70N.m;f<!@%.4LY1P3C)?QVG1Z8.ZUJnWC-?q$*/2&fSGYR\\]_qLF<T5+Heo!/H]FT5!E/ca.B%_9BibS*.)]Po%*S?QFo(kaGS5[.0N`87\\%plm?$*Tnq;dB@DBfTN.,Y:=;h?<1].eO0.R\\0F_Bfd%=JR5M>_1(/

In [177]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)
#training loop
for i in range(100000):
    print(f"step {i}: loss {loss.item():.4f}")
    xb, yb = get_batch('train')
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
# plotting loss
import matplotlib.pyplot as plt



step 0: loss 2.4456


IndexError: index out of range in self

In [116]:
logits.shape
print(decode(m.generate(xb, max_new_tokens=100)[0].tolist()))

. “If I’t domend brevey. — Hak y se f hon’thrn in — s pldno bere, is?” toresple s, tas.  hevid mbare ve e hi


In [ ]:
#self attention


torch.manual_seed(42)
B,T,C = 4,8,32
x1=torch.randn(B,T,C) # (B,T,C)

head_size = 16
key=nn.Linear(C, head_size, bias=False) # (C,head_size)
query=nn.Linear(C, head_size, bias=False) # (C,head_size)
value=nn.Linear(C, head_size, bias=False) # (C,head_size)
k=key(x1) # (B,T,head_size)
q=query(x1) # (B,T,head_size)
weights = q @ k.transpose(-2, -1)  # (B, T, head_size) @ (B, head_size, T) -> (B, T, T)

tril= torch.tril(torch.ones((T, T))) # (T,T)
# weights = torch.zeros((T, T)) # (T,T)
weights=weights.masked_fill(tril==0, float('-inf')) # (T,T)
weights=F.softmax(weights, dim=-1) # (T,T)
output = torch.matmul(weights, x1) # (B,T,C)
print(output)



tensor([[[ 1.9269,  1.4873,  0.9007,  ...,  0.0418, -0.2516,  0.8599],
         [-0.7538, -0.4219, -0.0092,  ...,  1.5011, -1.0068,  1.2838],
         [ 1.4646,  0.9943,  1.5864,  ..., -0.3506,  0.5980,  0.6755],
         ...,
         [-0.4599,  0.2508, -0.0482,  ...,  0.4776, -0.3928,  0.8727],
         [ 1.5130,  0.9772,  1.4039,  ..., -0.4304,  0.5649,  0.6918],
         [-0.4334,  0.0063,  0.4008,  ..., -0.2636,  0.6562,  0.9794]],

        [[-0.6855,  0.5636, -1.5072,  ...,  1.1566,  0.2691, -0.0366],
         [ 0.0175, -0.1055, -1.0981,  ...,  0.6430,  0.6657, -0.4375],
         [ 0.0825, -0.3012,  0.1249,  ..., -0.0343, -0.2112, -0.2610],
         ...,
         [ 0.7233, -0.8095, -0.4075,  ...,  0.0813,  0.8639, -0.7859],
         [-0.4920,  0.2737, -1.0159,  ...,  0.9975,  0.4399, -0.2065],
         [ 0.7317, -0.7136, -0.4925,  ...,  0.1837,  0.9935, -0.8176]],

        [[-1.3638,  0.1930, -0.6103,  ...,  0.6110,  1.2208, -0.6076],
         [-1.7060, -0.0985, -1.3020,  ..., -0

In [158]:
import torch

if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("Using GPU")
else:
    device = torch.device("cpu")

Using GPU


In [159]:
!pip install tiktoken

  Using cached regex-2024.11.6-cp311-cp311-macosx_11_0_arm64.whl.metadata (40 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 6.5 MB/s eta 0:00:00 MB/s eta 0:00:01
Using cached regex-2024.11.6-cp311-cp311-macosx_11_0_arm64.whl (284 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [tiktoken]


In [160]:
import tiktoken
enc = tiktoken.get_encoding("o200k_base")
assert enc.decode(enc.encode("hello world")) == "hello world"

# To get the tokeniser corresponding to a specific model in the OpenAI API:
enc = tiktoken.encoding_for_model("gpt-4o")

In [164]:
type(enc.encode("hello world"))


list

In [178]:
import torch

# Check for MPS support
if torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

# Create a tensor and move to MPS
x = torch.randn(5, 5).to(device)
print(x)

tensor([[-1.2851, -0.2685, -0.3118,  1.0101,  1.9954],
        [-0.9788, -0.6320,  0.1928,  1.2019,  0.8037],
        [-1.2548, -0.4295,  0.9173, -0.7931, -0.2413],
        [ 0.2915,  0.4070, -0.1934,  1.1370,  0.1862],
        [-0.1693, -0.3676,  0.0641, -1.6502,  0.6944]], device='mps:0')
